# Advanced Clustering Techniques with SUBMARIT

This notebook explores advanced clustering techniques and features available in SUBMARIT, including:
- Constrained clustering
- Multiple algorithm variants
- Entropy-based clustering
- GAP statistic for optimal cluster selection
- Top-k solution analysis
- Advanced parameter tuning

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Import SUBMARIT modules
from submarit.algorithms import (
    KSMLocalSearch, KSMLocalSearch2,
    KSMLocalSearchConstrained, KSMLocalSearchConstrained2
)
from submarit.evaluation import (
    GAPStatistic, EntropyClusterer, 
    ClusterEvaluator, EvaluationVisualizer
)
from submarit.validation import (
    run_clusters_topk, analyze_solution_stability,
    create_switching_matrix_distribution, k_sm_empirical_p
)

# Set style and random seed
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)

print("Libraries imported successfully!")

## 1. Constrained Clustering

Sometimes you need to enforce constraints on cluster sizes or membership. SUBMARIT provides constrained variants of the local search algorithms.

In [ ]:
# Generate a more complex substitution matrix
n_products = 20
n_true_clusters = 4

# Create block-diagonal structure with noise
substitution_matrix = np.zeros((n_products, n_products))
cluster_sizes = [5, 5, 6, 4]
start_idx = 0

for size in cluster_sizes:
    end_idx = start_idx + size
    # High within-cluster substitution
    substitution_matrix[start_idx:end_idx, start_idx:end_idx] = np.random.uniform(0.6, 0.9, (size, size))
    start_idx = end_idx

# Add noise and between-cluster substitution
noise = np.random.uniform(0, 0.3, (n_products, n_products))
substitution_matrix += noise
substitution_matrix = (substitution_matrix + substitution_matrix.T) / 2  # Symmetrize
np.fill_diagonal(substitution_matrix, 0)
substitution_matrix = np.clip(substitution_matrix, 0, 1)

# Visualize
plt.figure(figsize=(8, 6))
sns.heatmap(substitution_matrix, cmap='YlOrRd', cbar_kws={'label': 'Substitution Score'})
plt.title('Complex Substitution Matrix with 4 Submarkets')
plt.show()

In [ ]:
# Example 1: Minimum cluster size constraint
print("=== Constrained Clustering with Minimum Cluster Size ===")

# Define constraints
min_cluster_size = 3  # Each cluster must have at least 3 products

# Initialize constrained local search
constrained_search = KSMLocalSearchConstrained(
    n_clusters=4,
    min_cluster_size=min_cluster_size,
    max_iterations=200,
    n_restarts=20,
    random_state=42
)

# Run clustering
constrained_result = constrained_search.fit(substitution_matrix)

# Check cluster sizes
cluster_sizes = [np.sum(constrained_result.labels == i) for i in range(4)]
print(f"\nCluster sizes: {cluster_sizes}")
print(f"All clusters meet minimum size constraint: {all(size >= min_cluster_size for size in cluster_sizes)}")

# Visualize constrained clustering
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Cluster sizes bar chart
ax1.bar(range(4), cluster_sizes, color=['red', 'blue', 'green', 'orange'])
ax1.axhline(y=min_cluster_size, color='black', linestyle='--', label=f'Min size = {min_cluster_size}')
ax1.set_xlabel('Cluster')
ax1.set_ylabel('Number of Products')
ax1.set_title('Cluster Sizes with Constraint')
ax1.legend()

# Reordered matrix
sorted_indices = np.argsort(constrained_result.labels)
reordered_matrix = substitution_matrix[sorted_indices][:, sorted_indices]
sns.heatmap(reordered_matrix, ax=ax2, cmap='YlOrRd')
ax2.set_title('Reordered Matrix (Constrained Clustering)')

plt.tight_layout()
plt.show()

In [ ]:
# Example 2: Must-link and cannot-link constraints
print("=== Constrained Clustering with Must-Link/Cannot-Link ===")

# Define pairwise constraints
must_link = [(0, 1), (0, 2), (10, 11)]  # These products must be in the same cluster
cannot_link = [(0, 15), (5, 10)]  # These products cannot be in the same cluster

# Initialize constrained search with pairwise constraints
constrained_search2 = KSMLocalSearchConstrained2(
    n_clusters=4,
    must_link=must_link,
    cannot_link=cannot_link,
    max_iterations=200,
    n_restarts=20,
    random_state=42
)

# Run clustering
constrained_result2 = constrained_search2.fit(substitution_matrix)

# Verify constraints are satisfied
print("\nConstraint Verification:")
print("Must-link constraints:")
for i, j in must_link:
    same_cluster = constrained_result2.labels[i] == constrained_result2.labels[j]
    print(f"  Products {i} and {j}: {'✓ Same cluster' if same_cluster else '✗ Different clusters'}")

print("\nCannot-link constraints:")
for i, j in cannot_link:
    different_cluster = constrained_result2.labels[i] != constrained_result2.labels[j]
    print(f"  Products {i} and {j}: {'✓ Different clusters' if different_cluster else '✗ Same cluster'}")

## 2. Entropy-Based Clustering

Entropy-based clustering maximizes the information content of the clustering solution.

In [ ]:
# Initialize entropy-based clusterer
entropy_clusterer = EntropyClusterer(
    n_clusters=4,
    entropy_weight=0.5,  # Balance between cohesion and entropy
    max_iterations=200,
    random_state=42
)

# Run entropy-based clustering
print("Running entropy-based clustering...")
entropy_result = entropy_clusterer.fit(substitution_matrix)

# Compare with standard clustering
standard_search = KSMLocalSearch(n_clusters=4, random_state=42)
standard_result = standard_search.fit(substitution_matrix)

# Calculate entropy for both solutions
def calculate_cluster_entropy(labels):
    """Calculate entropy of cluster distribution"""
    _, counts = np.unique(labels, return_counts=True)
    probs = counts / len(labels)
    return -np.sum(probs * np.log2(probs + 1e-10))

entropy_standard = calculate_cluster_entropy(standard_result.labels)
entropy_entropy_based = calculate_cluster_entropy(entropy_result.labels)

print(f"\nEntropy comparison:")
print(f"Standard clustering entropy: {entropy_standard:.4f}")
print(f"Entropy-based clustering entropy: {entropy_entropy_based:.4f}")
print(f"\nCluster balance (entropy-based):")
for i in range(4):
    size = np.sum(entropy_result.labels == i)
    print(f"  Cluster {i}: {size} products ({size/n_products*100:.1f}%)")

## 3. GAP Statistic for Optimal Number of Clusters

The GAP statistic helps determine the optimal number of clusters by comparing the clustering quality to a null reference distribution.

In [ ]:
# Initialize GAP statistic calculator
gap_calculator = GAPStatistic(
    max_clusters=8,
    n_references=20,  # Number of reference datasets
    random_state=42
)

# Calculate GAP statistic
print("Calculating GAP statistic (this may take a moment)...")
gap_results = gap_calculator.compute(substitution_matrix)

# Plot GAP statistic
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# GAP values
k_values = gap_results.k_values
gap_values = gap_results.gap_values
gap_std = gap_results.gap_std

ax1.errorbar(k_values, gap_values, yerr=gap_std, marker='o', capsize=5)
ax1.axvline(x=gap_results.optimal_k, color='red', linestyle='--', 
            label=f'Optimal k = {gap_results.optimal_k}')
ax1.set_xlabel('Number of Clusters (k)')
ax1.set_ylabel('GAP Statistic')
ax1.set_title('GAP Statistic Analysis')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Within-cluster sum of squares
ax2.plot(k_values, gap_results.wss_values, marker='o')
ax2.set_xlabel('Number of Clusters (k)')
ax2.set_ylabel('Within-Cluster Sum of Squares')
ax2.set_title('Elbow Method')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nOptimal number of clusters: {gap_results.optimal_k}")
print(f"GAP value at optimal k: {gap_results.gap_values[gap_results.optimal_k-1]:.4f}")

## 4. Top-K Solution Analysis

Instead of just finding the best solution, we can analyze the top-k solutions to understand solution stability and alternative clusterings.

In [ ]:
# Run top-k analysis
print("Finding top-10 clustering solutions...")

topk_results = run_clusters_topk(
    substitution_matrix,
    n_clusters=4,
    k=10,  # Find top 10 solutions
    n_runs=50,  # Total number of runs
    algorithm='KSMLocalSearch',
    random_state=42
)

# Display top solutions
print("\nTop 10 Solutions:")
print("Rank | Objective Value | Frequency | Cumulative Freq")
print("-" * 50)
for i, solution in enumerate(topk_results.top_solutions):
    print(f"{i+1:4d} | {solution.objective:14.6f} | {solution.frequency:9d} | {solution.cumulative_frequency:15d}")

# Analyze solution stability
stability_analysis = analyze_solution_stability(topk_results)

print(f"\nSolution Stability Analysis:")
print(f"  Diversity score: {stability_analysis.diversity_score:.4f}")
print(f"  Top solution dominance: {stability_analysis.top_solution_dominance:.2%}")
print(f"  Average pairwise similarity: {stability_analysis.avg_pairwise_similarity:.4f}")

In [ ]:
# Visualize solution diversity
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. Objective value distribution
ax = axes[0, 0]
objectives = [sol.objective for sol in topk_results.top_solutions]
ax.bar(range(1, len(objectives)+1), objectives, color='steelblue')
ax.set_xlabel('Solution Rank')
ax.set_ylabel('Objective Value')
ax.set_title('Top-K Solution Objectives')

# 2. Frequency distribution
ax = axes[0, 1]
frequencies = [sol.frequency for sol in topk_results.top_solutions]
ax.bar(range(1, len(frequencies)+1), frequencies, color='darkgreen')
ax.set_xlabel('Solution Rank')
ax.set_ylabel('Frequency')
ax.set_title('Solution Frequency Distribution')

# 3. Pairwise similarity heatmap
ax = axes[1, 0]
n_solutions = min(5, len(topk_results.top_solutions))  # Show top 5
similarity_matrix = np.zeros((n_solutions, n_solutions))

for i in range(n_solutions):
    for j in range(n_solutions):
        if i != j:
            labels_i = topk_results.top_solutions[i].labels
            labels_j = topk_results.top_solutions[j].labels
            # Calculate Rand index as similarity
            from submarit.validation import RandIndex
            rand_calc = RandIndex()
            similarity_matrix[i, j] = rand_calc.compute(labels_i, labels_j).rand_index
        else:
            similarity_matrix[i, j] = 1.0

sns.heatmap(similarity_matrix, annot=True, fmt='.3f', cmap='YlGnBu', ax=ax,
            xticklabels=[f'Sol {i+1}' for i in range(n_solutions)],
            yticklabels=[f'Sol {i+1}' for i in range(n_solutions)])
ax.set_title('Pairwise Solution Similarity (Rand Index)')

# 4. Cluster assignment variability
ax = axes[1, 1]
# Calculate how often each product changes clusters across top solutions
assignment_variability = np.zeros(n_products)
for i in range(n_products):
    assignments = [sol.labels[i] for sol in topk_results.top_solutions[:5]]
    assignment_variability[i] = len(set(assignments)) / len(assignments)

ax.bar(range(n_products), assignment_variability, color='coral')
ax.set_xlabel('Product')
ax.set_ylabel('Assignment Variability')
ax.set_title('Product Clustering Stability (Top 5 Solutions)')
ax.set_xticks(range(0, n_products, 2))
ax.set_xticklabels([f'P{i}' for i in range(0, n_products, 2)])

plt.tight_layout()
plt.show()

## 5. Algorithm Comparison

Let's compare the performance of different algorithm variants:

In [ ]:
# Define algorithms to compare
algorithms = {
    'KSMLocalSearch': KSMLocalSearch(n_clusters=4, random_state=42),
    'KSMLocalSearch2': KSMLocalSearch2(n_clusters=4, random_state=42),
    'KSMLocalSearchConstrained': KSMLocalSearchConstrained(n_clusters=4, min_cluster_size=3, random_state=42),
    'EntropyClusterer': EntropyClusterer(n_clusters=4, random_state=42)
}

# Run comparison
comparison_results = {}
evaluator = ClusterEvaluator()

print("Comparing algorithms...")
for name, algorithm in algorithms.items():
    print(f"\nRunning {name}...")
    
    # Time the execution
    import time
    start_time = time.time()
    result = algorithm.fit(substitution_matrix)
    execution_time = time.time() - start_time
    
    # Evaluate
    eval_result = evaluator.evaluate(substitution_matrix, result.labels)
    
    comparison_results[name] = {
        'result': result,
        'eval': eval_result,
        'time': execution_time
    }
    
    print(f"  Objective: {result.best_objective:.6f}")
    print(f"  Silhouette: {eval_result.silhouette_score:.4f}")
    print(f"  Time: {execution_time:.3f}s")

In [ ]:
# Create comparison visualization
metrics = ['Objective', 'Silhouette', 'Davies-Bouldin', 'Time (s)']
algorithms_names = list(comparison_results.keys())

# Prepare data for plotting
data = []
for algo in algorithms_names:
    res = comparison_results[algo]
    data.append([
        res['result'].best_objective,
        res['eval'].silhouette_score,
        -res['eval'].davies_bouldin_index,  # Negative because lower is better
        res['time']
    ])

data = np.array(data).T

# Normalize data for radar chart (0-1 scale)
data_norm = np.zeros_like(data)
for i in range(len(metrics)):
    if metrics[i] == 'Time (s)':
        # For time, lower is better, so invert
        data_norm[i] = 1 - (data[i] - data[i].min()) / (data[i].max() - data[i].min() + 1e-10)
    else:
        data_norm[i] = (data[i] - data[i].min()) / (data[i].max() - data[i].min() + 1e-10)

# Create radar chart
angles = np.linspace(0, 2 * np.pi, len(metrics), endpoint=False).tolist()
data_norm = np.concatenate((data_norm, data_norm[:1]))  # Complete the circle
angles += angles[:1]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), subplot_kw=dict(projection='polar'))

# Radar chart
colors = ['red', 'blue', 'green', 'orange']
for idx, algo in enumerate(algorithms_names):
    values = data_norm[:, idx]
    ax1.plot(angles, values, 'o-', linewidth=2, label=algo, color=colors[idx])
    ax1.fill(angles, values, alpha=0.15, color=colors[idx])

ax1.set_xticks(angles[:-1])
ax1.set_xticklabels(metrics)
ax1.set_ylim(0, 1)
ax1.set_title('Algorithm Performance Comparison\n(Higher is Better)', pad=20)
ax1.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))

# Bar chart for execution times
ax2 = plt.subplot(1, 2, 2)
times = [comparison_results[algo]['time'] for algo in algorithms_names]
bars = ax2.bar(algorithms_names, times, color=colors)
ax2.set_ylabel('Execution Time (seconds)')
ax2.set_title('Algorithm Speed Comparison')
ax2.set_xticklabels(algorithms_names, rotation=45, ha='right')

# Add value labels on bars
for bar, time in zip(bars, times):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
             f'{time:.3f}s', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 6. Statistical Significance Testing

When comparing clustering solutions, it's important to test for statistical significance.

In [ ]:
# Create empirical distribution for significance testing
print("Creating empirical distribution for significance testing...")

# Generate null distribution
null_distribution = create_switching_matrix_distribution(
    substitution_matrix,
    n_simulations=1000,
    n_clusters=4,
    random_state=42
)

# Calculate p-values for our solutions
print("\nStatistical Significance Testing:")
print("Algorithm | Objective | P-value | Significant?")
print("-" * 50)

for name, res in comparison_results.items():
    p_value = k_sm_empirical_p(
        res['result'].best_objective,
        null_distribution
    )
    significant = p_value < 0.05
    print(f"{name:20s} | {res['result'].best_objective:.6f} | {p_value:.4f} | {'Yes' if significant else 'No'}")

# Visualize null distribution
plt.figure(figsize=(10, 5))
plt.hist(null_distribution.values, bins=50, alpha=0.7, color='gray', edgecolor='black')

# Add lines for our solutions
for name, res in comparison_results.items():
    plt.axvline(x=res['result'].best_objective, 
                label=name, linewidth=2, linestyle='--')

plt.xlabel('Objective Value')
plt.ylabel('Frequency')
plt.title('Null Distribution vs. Observed Results')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 7. Advanced Parameter Tuning

Let's explore how different parameters affect clustering performance:

In [ ]:
# Parameter grid for tuning
param_grid = {
    'n_restarts': [5, 10, 20, 50],
    'max_iterations': [50, 100, 200, 500],
    'initialization': ['random', 'kmeans++', 'spectral']
}

# Simple grid search (subset for demonstration)
print("Performing parameter tuning...")
results_grid = []

for n_restarts in [5, 20]:
    for max_iter in [100, 300]:
        for init in ['random', 'kmeans++']:
            # Run clustering
            search = KSMLocalSearch(
                n_clusters=4,
                n_restarts=n_restarts,
                max_iterations=max_iter,
                initialization=init,
                random_state=42
            )
            
            result = search.fit(substitution_matrix)
            eval_res = evaluator.evaluate(substitution_matrix, result.labels)
            
            results_grid.append({
                'n_restarts': n_restarts,
                'max_iterations': max_iter,
                'initialization': init,
                'objective': result.best_objective,
                'silhouette': eval_res.silhouette_score,
                'iterations_used': result.n_iterations
            })

# Convert to DataFrame for analysis
df_results = pd.DataFrame(results_grid)

# Find best parameters
best_idx = df_results['objective'].idxmax()
best_params = df_results.iloc[best_idx]

print("\nBest parameters found:")
print(f"  n_restarts: {best_params['n_restarts']}")
print(f"  max_iterations: {best_params['max_iterations']}")
print(f"  initialization: {best_params['initialization']}")
print(f"  Objective: {best_params['objective']:.6f}")
print(f"  Silhouette: {best_params['silhouette']:.4f}")

In [ ]:
# Visualize parameter effects
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. Effect of n_restarts
ax = axes[0, 0]
for init in df_results['initialization'].unique():
    data = df_results[df_results['initialization'] == init]
    grouped = data.groupby('n_restarts')['objective'].mean()
    ax.plot(grouped.index, grouped.values, marker='o', label=init)
ax.set_xlabel('Number of Restarts')
ax.set_ylabel('Average Objective')
ax.set_title('Effect of n_restarts on Performance')
ax.legend()
ax.grid(True, alpha=0.3)

# 2. Effect of max_iterations
ax = axes[0, 1]
for init in df_results['initialization'].unique():
    data = df_results[df_results['initialization'] == init]
    grouped = data.groupby('max_iterations')['objective'].mean()
    ax.plot(grouped.index, grouped.values, marker='s', label=init)
ax.set_xlabel('Max Iterations')
ax.set_ylabel('Average Objective')
ax.set_title('Effect of max_iterations on Performance')
ax.legend()
ax.grid(True, alpha=0.3)

# 3. Iterations actually used
ax = axes[1, 0]
pivot_data = df_results.pivot_table(
    values='iterations_used',
    index='max_iterations',
    columns='initialization',
    aggfunc='mean'
)
pivot_data.plot(kind='bar', ax=ax)
ax.set_xlabel('Max Iterations Allowed')
ax.set_ylabel('Average Iterations Used')
ax.set_title('Convergence Speed by Initialization')
ax.legend(title='Initialization')

# 4. Performance heatmap
ax = axes[1, 1]
pivot_objective = df_results.pivot_table(
    values='objective',
    index='n_restarts',
    columns='max_iterations',
    aggfunc='max'
)
sns.heatmap(pivot_objective, annot=True, fmt='.4f', cmap='YlOrRd', ax=ax)
ax.set_title('Best Objective by Parameters')

plt.tight_layout()
plt.show()

## Summary and Best Practices

### Key Takeaways:

1. **Constrained Clustering**:
   - Use when you have domain knowledge about cluster requirements
   - Helps ensure practical, actionable results
   - May sacrifice some objective value for constraint satisfaction

2. **Entropy-Based Clustering**:
   - Balances cluster cohesion with information content
   - Useful when cluster balance is important
   - Can help avoid trivial solutions

3. **GAP Statistic**:
   - Provides principled way to select number of clusters
   - Compare against elbow method for validation
   - Computationally intensive but robust

4. **Top-K Analysis**:
   - Reveals solution landscape and stability
   - Helps identify robust vs. unstable clusterings
   - Useful for understanding alternative interpretations

5. **Statistical Testing**:
   - Always test significance of clustering results
   - Use empirical distributions for p-value calculation
   - Consider multiple testing corrections

6. **Parameter Tuning**:
   - More restarts generally improve results but increase computation
   - Initialization method can significantly affect convergence
   - Balance computational cost with solution quality

### Recommended Workflow:

1. Start with GAP statistic to determine optimal number of clusters
2. Run multiple algorithms and compare results
3. Use top-k analysis to assess solution stability
4. Apply constraints if domain knowledge is available
5. Validate results with statistical significance testing
6. Fine-tune parameters based on specific needs